# Adding Housing Construction Materials

Based on a discussion on the performance of the model without the construction materials features, we are testing out using the materials but converting from municipality to grid values based on weights computed from Google Building Footprint data. Will look at the performance of the model after adding these.

In [1]:
%load_ext jupyter_black
import pandas as pd
import geopandas as gpd
from pathlib import Path
import os

In [16]:
base_dir = Path(os.getenv("STORM_DATA_DIR")) / "analysis/02_new_model_input/"
input_dir = base_dir / "05_vulnerablility/input/"
output_dir = base_dir / "05_vulnerablility/output/"
weights_dir = base_dir / "02_housing_damage/input/Google Footprint Data/"
hu_dir = base_dir / "02_housing_damage/output/"

In [17]:
mun_to_grid_weights = pd.read_csv(weights_dir / "ggl_mun_to_grid_weights.csv")
construction_materials_df = pd.read_excel(
    input_dir
    / "180814_construction-materials-of-the-outer-walls-and-roof_by-city_municipality.xlsx",
    sheet_name="by category",
)
hu_bygrid = pd.read_csv(hu_dir / "transformed_housingunits_bygrid.csv")

In [18]:
construction_materials_df.columns

Index(['Region', 'Region Code', 'Province', 'Province Code',
       'Municipality_City', 'Municipality_City Code', 'Housing Units',
       'Strong Roof/Strong Wall', 'Strong Roof/Light Wall',
       'Strong Roof/Salvage Wall', 'Light Roof/Strong Wall',
       'Light Roof/Light Wall', 'Light Roof/Salvage Wall',
       'Salvaged Roof/Strong Wall', 'Salvaged Roof/Light Wall',
       'Salvaged Roof/Salvage Wall'],
      dtype='object')

In [35]:
construction_materials_df["Strong Roof/Strong Wall"].sum()

14153053

In [19]:
merged_df = construction_materials_df.merge(
    mun_to_grid_weights[["ADM3_PCODE", "id", "Centroid", "weight"]],
    how="right",
    left_on="Municipality_City Code",
    right_on="ADM3_PCODE",
)

In [20]:
# There are 2 ways of going about this conversion to grid values
# 1. Convert the mun values to grid values using the weights and compute the % from the grid HUs
# 2. Compute % values by municipality then convert those values using the weights to grid values.

In [21]:
# Method 1
# merged_df.loc[:, merged_df.columns.str.contains("Roof")]
merged_df.drop(
    [
        "Region",
        "Region Code",
        "Province",
        "Province Code",
        "Municipality_City",
        "Housing Units",
    ],
    inplace=True,
    axis=1,
)

In [30]:
merged_df[
    merged_df.columns[merged_df.columns.str.contains("Roof")]
] = merged_df.loc[:, merged_df.columns.str.contains("Roof")].multiply(
    merged_df["weight"], axis="index"
)

In [31]:
merged_grid_df = merged_df.groupby(["id", "Centroid"]).sum().reset_index()

C:\Users\pauni\AppData\Local\Temp\ipykernel_27268\1508198058.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  merged_grid_df = merged_df.groupby(["id", "Centroid"]).sum().reset_index()


In [34]:
merged_grid_df["Strong Roof/Strong Wall"].sum()

14153052.999999996

In [38]:
merged_grid_df.drop("weight", inplace=True, axis=1)

In [40]:
hu_conmat_df = hu_bygrid.merge(merged_grid_df, on=["id", "Centroid"])

In [41]:
hu_conmat_df

,id,Centroid,numbuildings,Number of Household,hu_bygrid,Strong Roof/Strong Wall,Strong Roof/Light Wall,Strong Roof/Salvage Wall,Light Roof/Strong Wall,Light Roof/Light Wall,Light Roof/Salvage Wall,Salvaged Roof/Strong Wall,Salvaged Roof/Light Wall,Salvaged Roof/Salvage Wall
0,101,114.3E_11.1N,1,31.0,31.000000,7.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,4475,116.9E_7.9N,3,8445.0,3.301020,0.087127,1.392465,0.000000,0.014521,1.800235,0.001177,0.000000,0.000785,0.000000
2,4639,117.0E_8.2N,11,8445.0,12.103741,0.319466,5.105704,0.000000,0.053244,6.600863,0.004317,0.000000,0.002878,0.000000
3,4640,117.0E_8.1N,587,8445.0,645.899660,17.047881,272.458922,0.000000,2.841313,352.246075,0.230377,0.000000,0.153585,0.000000
4,4641,117.0E_8.0N,974,8445.0,1071.731293,28.287284,452.086866,0.000000,4.714547,584.476452,0.382261,0.000000,0.254840,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3721,20677,126.6E_7.6N,7393,13355.0,4449.357133,1991.623394,2069.140665,32.214970,7.718170,282.887704,7.382597,1.677863,15.436340,29.194816
3722,20678,126.6E_7.5N,2528,13355.0,1521.435795,681.025827,707.532477,11.015751,2.639190,96.732059,2.524443,0.573737,5.278380,9.983024
3723,20679,126.6E_7.4N,1484,22594.0,930.647069,233.390634,332.686995,2.144011,9.186363,343.398737,1.034035,0.104171,4.484423,1.956503
3724,20680,126.6E_7.3N,2798,18905.0,1800.666044,302.457800,550.919141,0.387762,23.839915,907.475646,1.528452,0.000000,9.436144,0.380230
